## Multiclass Classification Using Logistic Regressoion
In this assignment, you will perform multiclass classification using logistic regression. You will use the logistic regression library from the ``sklearn`` library. Some of the important libraries you 'may' use include the following:

**LogisticRegression from sklearn.linear_model**

**classification_report from sklearn.metrics**

**confusion_matrix from sklearn.metrics**

You can consult documentation at: 
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

## Data Set Information:

The database consists of the multi-spectral values of pixels in 3x3 neighbourhoods in a satellite image, and the classification associated with the central pixel in each neighbourhood. The aim is to predict this classification, given the multi-spectral values. In the sample database, the class of a pixel is coded as a number.

The Landsat satellite data is one of the many sources of information available for a scene. The interpretation of a scene by integrating spatial data of diverse types and resolutions including multispectral and radar data, maps indicating topography, land use etc. is expected to assume significant importance with the onset of an era characterised by integrative approaches to remote sensing (for example, NASA's Earth Observing System commencing this decade). Existing statistical methods are ill-equipped for handling such diverse data types. Note that this is not true for Landsat MSS data considered in isolation (as in this sample database). This data satisfies the important requirements of being numerical and at a single resolution, and standard maximum-likelihood classification performs very well. Consequently, for this data, it should be interesting to compare the performance of other methods against the statistical approach.

One frame of Landsat MSS imagery consists of four digital images of the same scene in different spectral bands. Two of these are in the visible region (corresponding approximately to green and red regions of the visible spectrum) and two are in the (near) infra-red. Each pixel is a 8-bit binary word, with 0 corresponding to black and 255 to white. The spatial resolution of a pixel is about 80m x 80m. Each image contains 2340 x 3380 such pixels.

The database is a (tiny) sub-area of a scene, consisting of 82 x 100 pixels. Each line of data corresponds to a 3x3 square neighbourhood of pixels completely contained within the 82x100 sub-area. Each line contains the pixel values in the four spectral bands (converted to ASCII) of each of the 9 pixels in the 3x3 neighbourhood and a number indicating the classification label of the central pixel. The number is a code for the following classes:

Number Class

1 red soil

2 cotton crop

3 grey soil

4 damp grey soil

5 soil with vegetation stubble

6 mixture class (all types present)

7 very damp grey soil

NB. There are no examples with class 6 in this dataset.

In each line of data the four spectral values for the top-left pixel are given first followed by the four spectral values for the top-middle pixel and then those for the top-right pixel, and so on with the pixels read out in sequence left-to-right and top-to-bottom. Thus, the four spectral values for the central pixel are given by attributes 17,18,19 and 20. If you like you can use only these four attributes, while ignoring the others. This avoids the problem which arises when a 3x3 neighbourhood straddles a boundary.


https://archive.ics.uci.edu/ml/datasets/Statlog+(Landsat+Satellite)


## 1. Import the data and split it into train/validate/test

Import the file '186_satimage.csv' and split it into train/validate/test set with ratio ``70/15/15``

**note: use random_state=777 whereever needed**

**note: the first 36 column are features and the last column is the class label**


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('186_satimage.csv')

X = data.iloc[:, :36]  # Features
y = data.iloc[:, 36]   # Labels

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=777)
X_validate, X_test, y_validate, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=777)


## 2. Check the details of the features and labels

Check some basic statistics of the features and the label

In [2]:
feature_stats = X_train.describe()

label_stats = y_train.describe()

print("Statistics for Features (X):")
print(feature_stats)

print("\nStatistics for Label (y):")
print(label_stats)


Statistics for Features (X):
          0.117596     1.241362     1.184036     0.815302    -0.158561  \
count  4500.000000  4500.000000  4500.000000  4500.000000  4500.000000   
mean      0.014406     0.015656     0.015797     0.007583     0.012057   
std       1.004998     1.000351     0.995853     0.992859     1.003293   
min      -2.160832    -2.473310    -2.780894    -2.624275    -2.223275   
25%      -0.690878    -0.550421    -0.798429    -0.666362    -0.674739   
50%      -0.102897     0.148811     0.102692     0.021553    -0.084821   
75%       0.779075     0.848043     0.823588     0.497802     0.800057   
max       2.543020     2.333912     2.445605     3.619879     2.422333   

          1.256483     1.193546     0.818486    -0.141965     0.879481  ...  \
count  4500.000000  4500.000000  4500.000000  4500.000000  4500.000000  ...   
mean      0.014576     0.016137     0.006898     0.014029     0.016000  ...   
std       1.000237     0.997232     0.994270     1.002346     0.999

## 3. Train a logistic regression classifier and fine-tune the hyper parameters on the validation set

After all the fine tuning, report the best results on the validation set.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization parameter
    'max_iter': [300, 400, 500],          # Maximum number of iterations
}

logistic_regression = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=777)

grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

best_logistic_regression = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=777, **best_params)
best_logistic_regression.fit(X_train, y_train)

y_pred_validate = best_logistic_regression.predict(X_validate)
validation_accuracy = accuracy_score(y_validate, y_pred_validate)

print("Best Hyperparameters:", best_params)
print("Validation Accuracy:", validation_accuracy)


C:\Users\Faisal\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Faisal\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_ite

Best Hyperparameters: {'C': 100, 'max_iter': 400}
Validation Accuracy: 0.8495850622406639


C:\Users\Faisal\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 4. Do the final test on the test set

Do the final scoring on the test set. Report different measure and show the confusion matrix. Record your observations.

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

y_pred_test = best_logistic_regression.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred_test)

precision = precision_score(y_test, y_pred_test, average=None)
recall = recall_score(y_test, y_pred_test, average=None)
f1 = f1_score(y_test, y_pred_test, average=None)

confusion = confusion_matrix(y_test, y_pred_test)

print("Test Accuracy:", test_accuracy)
print("Precision for Each Class:", precision)
print("Recall for Each Class:", recall)
print("F1-Score for Each Class:", f1)
print("Confusion Matrix:")
print(confusion)

class_labels = [1, 2, 3, 4, 5, 7]  # Excluding class 6
print("\nClassification Report:")
print(classification_report(y_test, y_pred_test, labels=class_labels))


Test Accuracy: 0.8632124352331606
Precision for Each Class: [0.97285068 0.96491228 0.86528497 0.5625     0.79130435 0.82945736]
Recall for Each Class: [0.97285068 0.94827586 0.94886364 0.35643564 0.86666667 0.8699187 ]
F1-Score for Each Class: [0.97285068 0.95652174 0.90514905 0.43636364 0.82727273 0.84920635]
Confusion Matrix:
[[215   0   4   0   2   0]
 [  2 110   0   0   4   0]
 [  0   0 167   9   0   0]
 [  3   0  22  36   5  35]
 [  1   4   0   0  91   9]
 [  0   0   0  19  13 214]]

Classification Report:
              precision    recall  f1-score   support

           1       0.97      0.97      0.97       221
           2       0.96      0.95      0.96       116
           3       0.87      0.95      0.91       176
           4       0.56      0.36      0.44       101
           5       0.79      0.87      0.83       105
           7       0.83      0.87      0.85       246

    accuracy                           0.86       965
   macro avg       0.83      0.83      0.82      